In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = 'drive/My Drive/CS534 Group Project/Files/'

Mounted at /content/drive/


In [ ]:
from transformers import pipeline
model = "siebert/sentiment-roberta-large-english"
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=model, max_length=512, truncation=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
import pandas as pd
df = pd.read_csv(path + "RS_2022_Amazon_with_subreddits_no_stopwords.csv")

In [ ]:
print(df['Subreddit'][0])
print(df['Body'][0])

r/homestead
Upvotes for non amazon links for US shipping


In [ ]:
df['Sentiment'] = df['Body'].apply(lambda row: sentiment_analysis(row))

In [ ]:
df.to_csv("RS_2022_RoBERTa.csv")

In [ ]:
print(sentiment_analysis(df['Body'][0]))

[{'label': 'POSITIVE', 'score': 0.998379111289978}]


In [ ]:
import pandas as pd

amazon_reviews = pd.read_csv(path + "sentiment_analysis_labeled_amazon.csv")
amazon_reviews_R = pd.read_csv("amazon_reviews_RoBERTA.csv")

In [ ]:
def convert_from_label(entry):
  if entry == 'NEGATIVE':
    return 0
  else:
    return 1

amazon_reviews_R['Sentiment'] = amazon_reviews_R['label'].apply(convert_from_label)

In [ ]:
from sklearn import metrics

print(f"Accuracy: {metrics.accuracy_score(amazon_reviews['Sentiment'], amazon_reviews_R['Sentiment'])}")
print(f"Precision: {metrics.precision_score(amazon_reviews['Sentiment'], amazon_reviews_R['Sentiment'])}")
print(f"Recall: {metrics.recall_score(amazon_reviews['Sentiment'], amazon_reviews_R['Sentiment'])}")
print(f"F1: {metrics.f1_score(amazon_reviews['Sentiment'], amazon_reviews_R['Sentiment'])}")
print(f"MCC: {metrics.matthews_corrcoef(amazon_reviews['Sentiment'], amazon_reviews_R['Sentiment'])}")

Accuracy: 0.468
Precision: 0.09328358208955224
Recall: 0.5208333333333334
F1: 0.15822784810126583
MCC: -0.009910604794223862


## Data Filtering/Analysis Below

In [ ]:
import numpy as np
np.savetxt("subreddits.csv", df.Subreddit.unique(), fmt='%s')

In [ ]:
# Score distribution
print(df.Score.describe())

count    40460.000000
mean        23.076149
std        461.812168
min          0.000000
25%          1.000000
50%          2.000000
75%          6.000000
max      68214.000000
Name: Score, dtype: float64


In [ ]:
df_filtered = df[df['Score'] >= 6.0]
print(df_filtered.Score.describe())

count    10284.000000
mean        85.791132
std        913.151029
min          6.000000
25%          8.000000
50%         14.000000
75%         33.000000
max      68214.000000
Name: Score, dtype: float64


In [ ]:
df_filtered['Body'] = df_filtered['Body'].str.replace('[^a-zA-Z\s]+', '')
df_filtered['Body'] = df_filtered['Body'].str.replace('\s+', ' ')

<ipython-input-17-76ee780b499a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_filtered['Body'] = df_filtered['Body'].str.replace('[^a-zA-Z\s]+', '')
<ipython-input-17-76ee780b499a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Body'] = df_filtered['Body'].str.replace('[^a-zA-Z\s]+', '')
<ipython-input-17-76ee780b499a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_filtered['Body'] = df_filtered['Body'].str.replace('\s+', ' ')
<ipython-input-17-76ee780b499a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [ ]:
# Stop words to filter out
stop_words = ['a', 'an', 'and', 'about', 'are', 'as', 'at', 'be', 'but', 'by',
              'for', 'if', 'from', 'Hi', 'hi', 'Hey', 'hey', 'has', 'I', 'Im', 'i', 'im'
              'in', 'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or','such',
              'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to',
              'was', 'will', 'with', 'you', 'youve', 'youre', 'your']

def filter_stop_words(sentence):
  words = sentence.split()
  filtered = [word for word in words if word not in stop_words and len(word) <= 20]
  return " ".join(filtered)

In [ ]:
df_filtered['Body'].apply(filter_stop_words)

0                     Upvotes non amazon links US shipping
1        Please use thread discuss whatever youve been ...
6        everyone With lineup coming soon well also get...
16       all very new investor months want follow buy h...
20       This Christmas got amazon gift card already ha...
                               ...                        
40445    just got pretty cheap starter kit Amazon nib p...
40450    The price models has definitely changed over l...
40457    CANT STRESS ENOUGH SHOWER CHAIR HOT WATER HOT ...
40458    In busy forum main headaches webmasters deal f...
40459    Spoilers follow Theres nothing world believe V...
Name: Body, Length: 10284, dtype: object

In [ ]:
df_filtered['Body']

0             Upvotes for non amazon links for US shipping
1        Please use this thread to discuss whatever you...
6        Hey everyone With the lineup coming soon well ...
16       Hey all Im a very new investor months and I wa...
20       This Christmas I got a amazon gift card I alre...
                               ...                        
40445    I just got a pretty cheap starter kit from Ama...
40450    The price of models has definitely changed ove...
40457    I CANT STRESS ENOUGH SHOWER CHAIR HOT WATER HO...
40458    In a busy forum the main headaches of webmaste...
40459    Spoilers to follow VicioushttpsiimgurcomXdfFPF...
Name: Body, Length: 10284, dtype: object

In [ ]:
print(sentiment_analysis(":)"))
print(sentiment_analysis(":("))
print(sentiment_analysis(":D"))
print(sentiment_analysis(":-)"))

[{'label': 'POSITIVE', 'score': 0.9985442161560059}]
[{'label': 'NEGATIVE', 'score': 0.9979062080383301}]
[{'label': 'POSITIVE', 'score': 0.9986230134963989}]
[{'label': 'POSITIVE', 'score': 0.9985790252685547}]
